# Évaluation du modèle

Dans ce notebook, vous apprendrez :

- Qu'est-ce que l'évaluation d'un modèle ?
- Comment évaluer un modèle dans ML.NET
- Entraîner et évaluer des modèles avec la validation croisée
- Explicabilité du modèle
- Comment améliorer votre modèle

## Qu'est-ce que l'évaluation d'un modèle ?

L'entraînement est le processus d'application d'algorithmes à des données historiques afin de créer un modèle qui représente fidèlement ces données. Ce modèle est ensuite utilisé sur de nouvelles données pour faire des prédictions.

L'évaluation du modèle est le processus d'utilisation de métriques pour quantifier l'efficacité avec laquelle votre modèle a appris les motifs dans vos données et applique ces apprentissages à des données nouvelles et inconnues.

## Comment évaluer un modèle dans ML.NET

### Importation du package Nuget ML.NET



In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"

#r "nuget: Microsoft.ML.AutoML"
#r "nuget: Microsoft.Data.Analysis"



Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.21.1 Microsoft.ML.AutoML, 0.21.1

Loading extensions from `C:\Users\Administrateur.000\.nuget\packages\microsoft.ml.automl\0.21.1\interactive-extensions\dotnet\Microsoft.ML.AutoML.Interactive.dll`

Loading extensions from `C:\Users\Administrateur.000\.nuget\packages\microsoft.data.analysis\0.21.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `C:\Users\Administrateur.000\.nuget\packages\skiasharp\2.88.6\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

### Référencer les packages avec des instructions `using`


In [2]:
using System.Text.Json;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.FastTree;
using static Microsoft.ML.Transforms.OneHotEncodingEstimator;


### Télécharger ou localiser les données

Le code suivant essaie de localiser le fichier de données à quelques emplacements connus ou le télécharge à partir de l'emplacement GitHub connu.


In [3]:
using System;
using System.IO;
using System.Net;

// Passage en culture anglaise pour les nombres à virgule
System.Threading.Thread.CurrentThread.CurrentCulture = new System.Globalization.CultureInfo("en-US");

string EnsureDataSetDownloaded(string fileName)
{
    var filePath = Path.Combine(Directory.GetCurrentDirectory(), "data", fileName);

    if (!File.Exists(filePath))
    {
        filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
    }

    if (!File.Exists(filePath))
    {
        using (var client = new WebClient())
        {
            client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
        }
        Console.WriteLine($"Downloaded {fileName} to : {filePath}");
    }
    else
    {
        Console.WriteLine($"{fileName} found here: {filePath}");
    }

    return filePath;
}

var trainDataPath = EnsureDataSetDownloaded("taxi-fare.csv");
var df = DataFrame.LoadCsv(trainDataPath);



taxi-fare.csv found here: e:\Dev\AI\Cours\CoursIA\MyIA.AI.Notebooks\ML\taxi-fare.csv


Une fois les données chargées, utilisez la méthode `Head` pour prévisualiser les cinq premières lignes.


In [4]:
df.Head(5)


index,vendor_id,rate_code,passenger_count,trip_time_in_secs,trip_distance,payment_type,fare_amount
0,CMT,1,1,1271,3.8,CRD,17.5
1,CMT,1,1,474,1.5,CRD,8
2,CMT,1,1,637,1.4,CRD,8.5
3,CMT,1,1,181,0.6,CSH,4.5
4,CMT,1,1,661,1.1,CRD,8.5


### Initialiser MLContext

Toutes les opérations ML.NET commencent par la classe [MLContext](https://docs.microsoft.com/dotnet/api/microsoft.ml.mlcontext). Initialiser `mlContext` crée un nouvel environnement ML.NET qui peut être partagé entre les objets du workflow de création de modèles. C'est similaire, conceptuellement, à `DBContext` dans Entity Framework.


In [5]:
var mlContext = new MLContext();


### Diviser les données en ensembles d'entraînement, de validation et de test

L'ensemble de données original est divisé en trois sous-ensembles : entraînement, validation et test. L'ensemble **d'entraînement** est utilisé pour apprendre les motifs de vos données. L'ensemble **de validation** est utilisé pour optimiser les hyperparamètres du modèle. L'ensemble **de test** est utilisé pour évaluer les performances de votre modèle à l'aide de métriques d'évaluation pour la tâche de régression.

#### Pourquoi ne pas utiliser l'ensemble de données complet ?

Bien que fournir plus d'exemples à votre entraîneur soit généralement recommandé, vous ne voulez pas d'un modèle qui ne performe bien que sur des données historiques. Au lieu de cela, vous cherchez un modèle qui peut apprendre de ces données historiques et généraliser ou faire des prédictions précises sur de nouvelles données inconnues.

Certains problèmes courants rencontrés pendant l'entraînement sont le surapprentissage et le sous-apprentissage. Le sous-apprentissage signifie que l'entraîneur sélectionné n'est pas assez puissant pour ajuster l'ensemble de données d'entraînement et se traduit généralement par une perte élevée pendant l'entraînement et une faible métrique sur l'ensemble de test. Pour résoudre cela, vous devez soit sélectionner un modèle plus puissant, soit effectuer plus d'ingénierie des caractéristiques. Le surapprentissage est l'opposé, et se produit lorsque le modèle apprend trop bien les données d'entraînement. Cela se traduit généralement par une faible perte métrique pendant l'entraînement, mais une perte élevée sur l'ensemble de test.

Une bonne analogie pour ces concepts est de réviser pour un examen. Disons que vous connaissez les questions et réponses à l'avance. Après avoir révisé, vous passez l'examen et obtenez un score parfait. Bonne nouvelle ! Cependant, lorsque vous repassez l'examen avec les questions réorganisées et légèrement reformulées, vous obtenez un score plus bas. Cela suggère que vous avez mémorisé les réponses et n'avez pas vraiment appris les concepts testés. C'est un exemple de surapprentissage. Le sous-apprentissage est le contraire, où les matériaux d'étude que vous avez reçus ne représentent pas fidèlement ce sur quoi vous êtes évalué lors de l'examen. En conséquence, vous devez deviner les réponses car vous n'avez pas assez de connaissances pour répondre correctement.


In [8]:
var trainTestData = mlContext.Data.TrainTestSplit(df, testFraction: 0.2);
var validationTestData = mlContext.Data.TrainTestSplit(trainTestData.TestSet, testFraction: 0.5);

var trainSet = trainTestData.TrainSet;
var validationSet = validationTestData.TrainSet;
var testSet = validationTestData.TestSet;


### Créer le pipeline d'entraînement

Pour cet ensemble de données, les transformations suivantes sont appliquées :

- `OneHotEncoding` pour convertir les valeurs catégorielles en valeurs numériques
- `ReplaceMissingValues` pour remplacer les valeurs manquantes
- `Concatenate` prend toutes les caractéristiques et crée un vecteur de caractéristiques

AutoML est utilisé pour définir une expérience de régression en utilisant la colonne `fare_amount` comme colonne à prédire ou colonne d'étiquette.


In [9]:
var pipeline = 
    mlContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair("vendor_id", "vendor_id"), new InputOutputColumnPair("payment_type", "payment_type") }, outputKind: OutputKind.Binary)
        .Append(mlContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair("rate_code", "rate_code"), new InputOutputColumnPair("passenger_count", "passenger_count"), new InputOutputColumnPair("trip_time_in_secs", "trip_time_in_secs"), new InputOutputColumnPair("trip_distance", "trip_distance") }))
        .Append(mlContext.Transforms.Concatenate("Features", new[] { "vendor_id", "payment_type", "rate_code", "passenger_count", "trip_time_in_secs", "trip_distance" }))
        .Append(mlContext.Auto().Regression(labelColumnName: "fare_amount"));


### Configurer l'expérience

Utilisez AutoML pour configurer notre expérience pour entraîner pendant 60 secondes en utilisant le pipeline que vous venez de définir.

Par défaut, AutoML évalue les modèles qu'il entraîne en utilisant la métrique d'évaluation que vous souhaitez optimiser. Dans ce cas, il s'agit du R-carré, qui est calculé en comparant la valeur réelle `fare_amount` avec la valeur prédite `Score`.

Les métriques d'évaluation dépendent fortement de la tâche. Pour la régression, certaines métriques courantes incluent :

- Erreur absolue moyenne (MAE)
- Erreur quadratique moyenne (MSE)
- Racine de l'erreur quadratique moyenne (RMSE)
- R-carré

Votre ensemble de données et ce que vous essayez d'atteindre influencent fortement votre sélection de métrique. Si vous avez des valeurs aberrantes dans votre ensemble de données, elles peuvent fausser vos prédictions. MAE, MSE et RMSE calculent la distance entre les points de données prédits et réels. Toutes ces mesures sont sensibles aux valeurs aberrantes, donc si vous avez des valeurs aberrantes dans votre ensemble de données, elles apparaîtront dans vos métriques. Le R-carré calcule la corrélation entre les valeurs réelles et préd
ites. Cependant, à mesure que vous ajoutez plus de points de données, votre R-carré peut continuer à augmenter, donnant l'impression erronée qu'un modèle avec une valeur de R-carré élevée a de bonnes capacités prédictives. Le résultat d'une valeur élevée de R-carré peut parfois indiquer un surapprentissage.


In [10]:
var experiment = 
    mlContext.Auto().CreateExperiment()
        .SetPipeline(pipeline)
        .SetTrainingTimeInSeconds(60)
        .SetDataset(trainSet, validationSet)
        .SetRegressionMetric(RegressionMetric.RSquared, "fare_amount", "Score");


### Exécuter l'expérience


In [11]:
var result = await experiment.RunAsync();


### Voir les métriques d'évaluation du meilleur modèle


In [12]:
$"R-Squared: {result.Metric}"


R-Squared: 0,9512696891212781

Notez que pendant l'entraînement, les métriques d'évaluation ont été calculées en utilisant l'ensemble de validation. Pour voir comment votre modèle performe sur de nouvelles données, évaluez ses performances par rapport à l'ensemble de test.

Commencez par obtenir le meilleur modèle en utilisant la propriété `Model` des résultats d'entraînement. Ensuite, utilisez la méthode `Transform` pour utiliser le modèle afin de faire des prédictions sur l'ensemble de test.


In [13]:
ITransformer bestModel = result.Model;
var predictions = bestModel.Transform(testSet);


Inspectez les premières prédictions (colonne `Score`) et comparez-les avec la valeur réelle (colonne `fare_amount`). Ensuite, calculez la différence entre elles.


In [14]:
var actual = predictions.GetColumn<float>("fare_amount");
var predicted = predictions.GetColumn<float>("Score");

var compare = 
    actual
        .Zip(predicted, (actual, pred) => new { Actual = actual, Predicted = pred, Difference = actual - pred })
        .Take(5);

compare


index value 0 { Actual = 24,5, Predicted = 24,378075, Difference = 0,121925354 } Actual 24.5 Predicted 24.378075 Difference 0.121925354 1 { Actual = 9,5, Predicted = 9,176805, Difference = 0,32319546 } Actual 9.5 Predicted 9.176805 Difference 0.32319546 2 { Actual = 4,5, Predicted = 4,646393, Difference = -0,14639282 } Actual 4.5 Predicted 4.646393 Difference -0.14639282 3 { Actual = 8, Predicted = 8,146669, Difference = -0,14666939 } Actual 8 Predicted 8.146669 Difference -0.14666939 4 { Actual = 52, Predicted = 51,954372, Difference = 0,045627594 } Actual 52 Predicted 51.954372 Difference 0.045627594

Rien qu'en comparant rapidement les premières valeurs, vous pouvez voir que les prédictions sont généralement à quelques centimes près de la valeur réelle.

Avec ML.NET, vous n'avez pas besoin de calculer manuellement les métriques d'évaluation pour vos modèles. ML.NET fournit une méthode intégrée `Evaluate` pour chacune des tâches de machine learning qu'il prend en charge. Utilisez la méthode `Evaluate` pour la tâche de régression afin de calculer les métriques d'évaluation pour l'ensemble de test où la colonne `fare_amount` est la valeur réelle et la colonne `Score` est la valeur prédite.


In [15]:
var evaluationMetrics = mlContext.Regression.Evaluate(predictions, "fare_amount", "Score");


L'utilisation de la méthode `Evaluate` permet non seulement de calculer la métrique que vous avez optimisée pendant l'entraînement (R-carré), mais aussi toutes les métriques pour la tâche de régression.


In [16]:
evaluationMetrics


MeanAbsoluteError,0.40920465912875675
MeanSquaredError,5.150957594966312
RootMeanSquaredError,2.269572117154754
LossFunction,5.150957527947543
RSquared,0.944208547622762


### Utiliser le R-carré pour évaluer le modèle

Bien que vous ayez plusieurs métriques à choisir, lorsque vous avez entraîné le modèle, vous avez optimisé pour le R-carré. Le R-carré (R2), ou coefficient de détermination, représente la puissance prédictive du modèle avec une valeur comprise entre -inf et 1,00. 1,00 signifie qu'il y a un ajustement parfait, et l'ajustement peut être arbitrairement mauvais, donc les scores peuvent être négatifs. Un score de 0,00 signifie que le modèle devine la valeur attendue pour l'étiquette. Une valeur négative de R2 indique que l'ajustement ne suit pas la tendance des données et que le modèle performe moins bien que le hasard. Cela n'est possible qu'avec des modèles de régression non linéaire ou des régressions linéaires contraintes. Le R2 mesure à quel point les valeurs réelles des données de test sont proches des valeurs prédites.

Pour plus d'informations sur les autres métriques d'évaluation, consultez le guide sur [comment évaluer votre modèle ML.NET avec des métriques](https://docs.microsoft.com/dotnet/machine-learning/resources/metrics).

## Entraîner et évaluer des modèles en utilisant la validation croisée

### Qu'est-ce que la validation croisée ?

La validation croisée est une technique d'entraînement et d'évaluation de modèle qui divise les données en plusieurs partitions et entraîne plusieurs modèles sur ces partitions. Cette technique améliore la robustesse du modèle en réservant des données du processus d'entraînement. En plus d'améliorer les performances sur des observations non vues, dans des environnements de données limitées, elle peut être un outil efficace pour entraîner des modèles avec un plus petit ensemble de données.

### Entraîner un modèle en utilisant la validation croisée

Commencez par initialiser le `MLContext`.


In [17]:
var cvMLContext = new MLContext();


Ensuite, définissez votre pipeline. Dans ce cas, l'entraîneur réel est utilisé au lieu de `SweepableEstimator` d'AutoML.


In [18]:
var cvMLPipeline = 
    cvMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair("vendor_id", "vendor_id"), new InputOutputColumnPair("payment_type", "payment_type") }, outputKind: OutputKind.Binary)
        .Append(cvMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair("rate_code", "rate_code"), new InputOutputColumnPair("passenger_count", "passenger_count"), new InputOutputColumnPair("trip_time_in_secs", "trip_time_in_secs"), new InputOutputColumnPair("trip_distance", "trip_distance") }))
        .Append(cvMLContext.Transforms.Concatenate("Features", new[] { "vendor_id", "payment_type", "rate_code", "passenger_count", "trip_time_in_secs", "trip_distance" }))
        .Append(cvMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount"));


Utilisez la méthode `CrossValidate` pour démarrer l'entraînement et l'évaluation sur vos données en utilisant le pipeline défini. Par défaut, les données sont divisées en cinq sous-ensembles, mais vous pouvez définir cette valeur à n'importe quelle valeur de votre choix en utilisant le paramètre `numberOfFolds`.


In [19]:
var cvResults = cvMLContext.Regression.CrossValidate(trainSet, cvMLPipeline, labelColumnName: "fare_amount");

cvResults.Select(x => x.Metrics)


index value 0 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.3080861318665928 MeanSquaredError 10.437064100433819 RootMeanSquaredError 3.230644533283385 LossFunction 10.437063966978595 RSquared 0.8884018879298109 1 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.360165666717988 MeanSquaredError 10.798742155503342 RootMeanSquaredError 3.2861439645127146 LossFunction 10.798742212339462 RSquared 0.8850517190709868 2 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.2518280553341605 MeanSquaredError 8.216770508755435 RootMeanSquaredError 2.8664909748253935 LossFunction 8.216770540499184 RSquared 0.9086191464533752 3 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.3420262660438258 MeanSquaredError 10.229870802662239 RootMeanSquaredError 3.1984169213319014 LossFunction 10.229870780300226 RSquared 0.8886983637741152 4 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.5430782453416307 MeanSquaredError 10.122911443078698 RootMeanSquaredError 3.1816523133552317 LossFunction 10.122911478082717 RSquared 0.88835065401792

### Calculer les métriques d'évaluation de l'ensemble de test

Comme dans l'exemple précédent, utilisez la méthode `Evaluate` sur l'ensemble complet de test pour évaluer les performances des modèles entraînés en utilisant la validation croisée.


In [20]:
var cvTestEvalMetrics = 
    cvResults
        .Select(fold => fold.Model.Transform(trainTestData.TestSet))
        .Select(predictions => cvMLContext.Regression.Evaluate(predictions, "fare_amount", "Score"));
        
cvTestEvalMetrics


index value 0 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.2946164063640293 MeanSquaredError 9.741100529118015 RootMeanSquaredError 3.1210736180228134 LossFunction 9.741100436339362 RSquared 0.8944318265394658 1 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.3517783571125017 MeanSquaredError 10.697284288708751 RootMeanSquaredError 3.270670311833455 LossFunction 10.697284293348838 RSquared 0.8840692835505208 2 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.258738437439045 MeanSquaredError 10.056973626481687 RootMeanSquaredError 3.1712731869836897 LossFunction 10.056973680972417 RSquared 0.8910085843878911 3 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.3321059289142654 MeanSquaredError 10.119475327642087 RootMeanSquaredError 3.1811122783771855 LossFunction 10.119475305281489 RSquared 0.8903312286404623 4 Microsoft.ML.Data.RegressionMetrics MeanAbsoluteError 1.5373861941022984 MeanSquaredError 11.21522705311926 RootMeanSquaredError 3.3489143096112897 LossFunction 11.215227048598013 RSquared 0.878456132199452

## Explicabilité du modèle

Les métriques d'évaluation sont un bon moyen de quantifier la précision avec laquelle votre modèle fait des prédictions sur de nouvelles données. Cependant, une bonne métrique d'évaluation ne doit pas être le seul facteur à considérer lors de l'évaluation de votre modèle. Pour renforcer la confiance autour de votre modèle et des décisions qu'il prend, il est important de comprendre comment et pourquoi il prend ces décisions.

Les modèles deviennent de plus en plus courants dans la société et affectent la vie des individus. Par exemple, imaginons qu'un modèle de machine learning soit utilisé pour établir un diagnostic médical. Il est probable que le modèle ait raison dans son diagnostic, mais les enjeux d'un mauvais diagnostic sont élevés en raison des effets sur la santé d'un individu. Il est donc important pour tous les intervenants (patients, médecins, régulateurs) de comprendre ce qui a poussé un modèle à établir ce diagnostic afin de se sentir confiants dans sa décision.

### Explications globales et locales

Lors de l'explication des modèles de machine learning, vous pouvez le faire au niveau global et local.

Les explications globales sont des généralisations au niveau agrégé. Par exemple, disons que vous construisez un modèle pour prédire les tarifs de taxi. En expliquant pourquoi certains tarifs sont plus chers que d'autres, vous trouverez probablement que les trajets qui parcourent une distance plus longue ou qui durent plus longtemps sont probablement plus chers. Bien que cela ne vous dise pas exactement pourquoi un trajet particulier était plus cher qu'un autre, à un niveau agrégé, vous pouvez voir quelles caractéristiques sont importantes pour le modèle lorsqu'il prend ses décisions.

Si vous avez besoin de plus de granularité pour expliquer les décisions de votre modèle, c'est là que les explications locales entrent en jeu. Les explications locales vous permettent de voir pour une prédiction donnée quelles caractéristiques ont contribué à la décision du modèle. Par exemple, disons qu'un modèle est utilisé pour déterminer le risque de crédit pour des prêts personnels. Étant donné deux clients avec des montants de dette, des revenus et des historiques de paiement différents, le modèle détermine lequel des clients est le plus susceptible de rembourser le prêt. En utilisant des techniques d'explicabilité locale, vous pouvez inspecter au niveau individuel quelles caractéristiques ont contribué à la décision de refuser un prêt.

### Techniques d'explicabilité dans ML.NET

ML.NET fournit deux techniques pour expliquer les modèles :

- Importance des caractéristiques par permutation (PFI)
- Calcul de la contribution des caractéristiques (FCC)

#### Importance des caractéristiques par permutation (PFI)

L'importance des caractéristiques par permutation est une technique d'explicabilité **globale**. À un haut niveau, elle mélange aléatoirement les données une caractéristique à la fois pour l'ensemble de données complet et calcule dans quelle mesure la métrique de performance d'intérêt diminue. Plus le changement est important, plus la caractéristique est importante. Pour plus d'informations, consultez [Interpréter les prédictions de modèle en utilisant l'importance des caractéristiques par permutation](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/explain-machine-learning-model-permutation-feature-importance-ml-net).

#### Calcul de la contribution des caractéristiques (FCC)

Le calcul de la contribution des caractéristiques est une technique d'explicabilité **locale**. Cette technique calcule une liste spécifique au modèle des contributions par caractéristique à chacune des prédictions. Ces contributions peuvent être positives (elles augmentent la métrique d'évaluation) ou négatives (elles diminuent la métrique d'évaluation).

### Expliquer les modèles avec l'importance des caractéristiques par permutation (PFI)

#### Initialiser MLContext



In [21]:
var pfiMLContext = new MLContext();


#### Définir le pipeline de préparation des données


In [22]:
var pfiDataPipeline = 
    pfiMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair("vendor_id", "vendor_id"), new InputOutputColumnPair("payment_type", "payment_type") }, outputKind: OutputKind.Binary)
        .Append(pfiMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair("rate_code", "rate_code"), new InputOutputColumnPair("passenger_count", "passenger_count"), new InputOutputColumnPair("trip_time_in_secs", "trip_time_in_secs"), new InputOutputColumnPair("trip_distance", "trip_distance") }))
        .Append(pfiMLContext.Transforms.Concatenate("Features", new[] { "vendor_id", "payment_type", "rate_code", "passenger_count", "trip_time_in_secs", "trip_distance" }));


#### Appliquer les transformations de données aux données d'entraînement


In [23]:
var pfiPreprocessedData = 
    pfiDataPipeline
        .Fit(trainSet)
        .Transform(trainSet);


#### Définir votre entraîneur


In [24]:
var pfiTrainer = pfiMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount");


#### Ajuster l'entraîneur à vos données prétraitées


In [25]:
var pfiModel = pfiTrainer.Fit(pfiPreprocessedData);


#### Calculer l'importance des caractéristiques par permutation (PFI)


In [26]:
var permutationFeatureImportance =
    mlContext
        .Regression
        .PermutationFeatureImportance(pfiModel, pfiPreprocessedData, permutationCount: 3, labelColumnName: "fare_amount");


#### Extraire la métrique du R-carré


In [27]:
var pfiMetrics = 
    permutationFeatureImportance
        .Select((metric, idx) => new { idx, metric.RSquared })
        .OrderByDescending(x => Math.Abs(x.RSquared.Mean));


#### Obtenir la liste des noms de caractéristiques


In [28]:
var featureContributionColumn = pfiPreprocessedData.Schema.GetColumnOrNull("Features");
var slotNames = new VBuffer<ReadOnlyMemory<char>>();
featureContributionColumn.Value.GetSlotNames(ref slotNames);
var slotNameValues = slotNames.DenseValues();


#### Mapper les métriques PFI aux noms des caractéristiques


In [29]:
var featureImportance = 
    pfiMetrics
        .Zip(slotNameValues, (a, b) => new KeyValuePair<string, double>(b.ToString(), a.RSquared.Mean));

featureImportance


index value 0 [vendor_id.Bit2, -0,5100349451041303] Key vendor_id.Bit2 Value -0.5100349451041303 1 [vendor_id.Bit1, -0,2094975206996916] Key vendor_id.Bit1 Value -0.2094975206996916 2 [vendor_id.Bit0, -0,20499759578445162] Key vendor_id.Bit0 Value -0.20499759578445162 3 [payment_type.Bit3, -0,0014385602479098554] Key payment_type.Bit3 Value -0.0014385602479098554 4 [payment_type.Bit2, -0,0005410433692841776] Key payment_type.Bit2 Value -0.0005410433692841776 5 [payment_type.Bit1, -0,00016769409566042004] Key payment_type.Bit1 Value -0.00016769409566042004 6 [payment_type.Bit0, -7,467922472670363E-05] Key payment_type.Bit0 Value -7.467922472670363E-05 7 [rate_code, -5,266369401309608E-07] Key rate_code Value -5.266369401309608E-07 8 [passenger_count, 0] Key passenger_count Value 0 9 [trip_time_in_secs, 0] Key trip_time_in_secs Value 0 10 [trip_distance, 0] Key trip_distance Value 0

### Expliquer les modèles avec le calcul de la contribution des caractéristiques (FCC)

#### Initialiser MLContext


In [30]:
var fccMLContext = new MLContext();


#### Définir le pipeline de préparation des données


In [31]:
var fccDataPipeline = 
    fccMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair("vendor_id", "vendor_id"), new InputOutputColumnPair("payment_type", "payment_type") }, outputKind: OutputKind.Binary)
        .Append(fccMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair("rate_code", "rate_code"), new InputOutputColumnPair("passenger_count", "passenger_count"), new InputOutputColumnPair("trip_time_in_secs", "trip_time_in_secs"), new InputOutputColumnPair("trip_distance", "trip_distance") }))
        .Append(fccMLContext.Transforms.Concatenate("Features", new[] { "vendor_id", "payment_type", "rate_code", "passenger_count", "trip_time_in_secs", "trip_distance" }));


#### Appliquer les transformations de données aux données d'entraînement


In [32]:
var fccPreprocessedData = 
    fccDataPipeline
        .Fit(trainSet)
        .Transform(trainSet);


#### Définir votre entraîneur


In [33]:
var fccTrainer = fccMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount");


#### Ajuster l'entraîneur à vos données prétraitées


In [34]:
var fccModel = fccTrainer.Fit(fccPreprocessedData);


#### Calculer les contributions des caractéristiques


In [ ]:
var featureContributionCalc = 
    fccMLContext.Transforms.CalculateFeatureContribution(fccModel, normalize: false)
        .Fit(fccPreprocessedData)
        .Transform(fccPreprocessedData);


#### Obtenir la liste des noms de caractéristiques


In [35]:
var featureContributionColumn = featureContributionCalc.Schema.GetColumnOrNull("FeatureContributions");
var slotNames = new VBuffer<ReadOnlyMemory<char>>();
featureContributionColumn.Value.GetSlotNames(ref slotNames);
var slotNameValues = slotNames.DenseValues();


Error: (1,33): error CS0103: Le nom 'featureContributionCalc' n'existe pas dans le contexte actuel

#### Obtenir les valeurs de contribution des caractéristiques


In [ ]:
var featureContributionValues = featureContributionCalc.GetColumn<float[]>("FeatureContributions");


#### Mapper les valeurs de contribution des caractéristiques aux noms des caractéristiques


In [ ]:
var featureContributions = 
    featureContributionValues
        .Select(x => x.Zip(slotNameValues, (a, b) => new KeyValuePair<string, float>(b.ToString(), a)));


#### Afficher les contributions des caractéristiques pour la première prédiction


In [ ]:
featureContributions.First()


## Comment puis-je améliorer mon modèle ?

L'évaluation du modèle est une étape importante dans le workflow de machine learning pour déterminer si un modèle est prêt à être déployé en production. Si votre modèle ne répond pas à vos critères pour être prêt pour la production, il y a plusieurs choses que vous pouvez essayer pour améliorer votre modèle. Celles-ci incluent :

- **Reformuler le problème** - Essayez-vous de résoudre le bon problème ? Envisagez d'examiner le problème sous différents points de vue.
- **Fournir plus d'échantillons de données** - L'expérience est le meilleur enseignant. Fournir plus d'exemples qui représentent votre espace de problème aide les entraîneurs à identifier plus de cas particuliers.
- **Ajouter des caractéristiques (plus de contexte)** - Construire un contexte autour des points de données aide les algorithmes ainsi que les experts en la matière à mieux prendre des décisions. Par exemple, le fait qu'une maison ait trois chambres ne donne pas à lui seul une bonne indication de son prix. Cependant, si vous ajoutez du contexte et que vous savez maintenant qu'elle se trouve dans un quartier suburbain en dehors d'une grande métropole où l'âge moyen est de 38 ans, le revenu moyen des ménages est de 80 000 $ et les écoles sont dans le 20e percentile supérieur, alors l'algorithme a plus d'informations pour baser ses décisions.
- **Utiliser des données et des caractéristiques significatives** - Plus de données et de caractéristiques (contexte) peuvent aider à améliorer la précision, mais aussi introduire du bruit. Envisagez d'utiliser l'importance des caractéristiques par permutation (PFI) et le calcul de la contribution des caractéristiques (FCC) pour déterminer les caractéristiques impactant vos prédictions et supprimer celles qui ne contribuent pas à votre modèle.
- **Utiliser la validation croisée** - La validation croisée peut être un outil efficace pour entraîner des modèles avec des ensembles de données plus petits.
- **Ajustement des hyperparamètres** - Les hyperparamètres sont tout aussi importants que les paramètres appris pendant l'entraînement. Trouver les bons est un processus d'essais et d'erreurs. Utilisez les espaces de recherche AutoML et les estimateurs à balayer pour vous aider à trouver les bons hyperparamètres pour votre algorithme.
- **Choisir un algorithme différent** - Tout comme l'ajustement des hyperparamètres, trouver le bon algorithme pour entraîner votre modèle est un processus d'essais et d'erreurs. Utilisez AutoML pour vous aider à itérer à travers les différents algorithmes disponibles dans ML.NET pour vous aider à choisir l'algorithme optimal pour résoudre votre problème.

Pour plus d'informations, consultez le guide sur [comment améliorer votre modèle ML.NET](https://docs.microsoft.com/dotnet/machine-learning/resources/improve-machine-learning-model-ml-net).

## Continuer à apprendre

> [⏩ Module suivant - API AutoML Sweepable](https://github.com/dotnet/csharp-notebooks/blob/main/machine-learning/05%20-%20AutoML%20Sweepable%20API.ipynb)  
> [⏪ Module précédent - Entraînement et AutoML](https://github.com/dotnet/csharp-notebooks/blob/main/machine-learning/03-Training%20and%20AutoML.ipynb)

### Plus d'exemples de bout en bout

- [Classification binaire avec le jeu de données Titanic](./REF-Kaggle%20with%20Titanic%20Dataset.ipynb)  
- [Prédiction de valeurs/Régression avec le jeu de données Taxi](./E2E-Regression%20with%20Taxi%20Dataset.ipynb)
